In [40]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time

# URL 설정
base_url = "https://cointelegraph.com/tags/markets"
today = datetime.now()
one_week_ago = today - timedelta(days=14)

def create_driver():
    """Selenium WebDriver 생성 및 설정"""
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")  # Selenium 표시 제거
    driver = webdriver.Chrome(options=options)
    return driver

def scrape_articles():
    """
    Cointelegraph Markets 섹션에서 기사를 크롤링하는 함수.
    
    Returns:
        articles (list): 크롤링된 기사 목록 (제목, 링크, 날짜 포함)
    """
    driver = create_driver()
    driver.get(base_url)
    
    # 페이지 로딩 대기
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "li[data-testid='posts-listing__item']")))
    
    articles = []
    seen_titles = set()  # 중복 제거를 위한 제목 저장소
    
    # footer 요소 가져오기 (스크롤 제한용)
    footer_xpath = "//footer"
    footer_element = driver.find_element(By.XPATH, footer_xpath)
    
    try:
        while True:
            # 기사 추출
            elements = driver.find_elements(By.CSS_SELECTOR, "li[data-testid='posts-listing__item']")
            new_articles_found = False
            
            for element in elements:
                try:
                    # 제목과 링크 추출
                    title_tag = element.find_element(By.CLASS_NAME, "post-card-inline__title-link")
                    title = title_tag.text.strip()
                    link = title_tag.get_attribute("href")
                    
                    # 날짜 추출
                    date_tag = element.find_element(By.TAG_NAME, "time")
                    date_str = date_tag.get_attribute("datetime")
                    article_date = datetime.strptime(date_str[:10], '%Y-%m-%d')
                    
                    # 날짜 필터링 (일주일 이내 기사만)
                    if one_week_ago <= article_date <= today and title not in seen_titles:
                        articles.append({
                            "title": title,
                            "link": link,
                            "date": article_date.strftime('%Y-%m-%d')
                        })
                        seen_titles.add(title)  # 제목 저장하여 중복 방지
                        new_articles_found = True
                        print(f"기사 저장: {title} ({article_date.strftime('%Y-%m-%d')})")
                    
                    # 일주일 범위를 벗어난 기사 발견 시 종료
                    elif article_date < one_week_ago:
                        print("일주일 범위를 벗어난 기사가 발견되었습니다. 크롤링을 종료합니다.")
                        driver.quit()
                        return articles

                except Exception as e:
                    print(f"Error processing article: {e}")
            
            # 현재 스크롤 위치 확인
            current_scroll_position = driver.execute_script("return window.scrollY + window.innerHeight")
            footer_position = footer_element.location['y']  # footer의 Y축 위치
            
            # footer에 도달하기 전에만 스크롤 수행
            if current_scroll_position + 100 >= footer_position:
                print("Reached near the footer. Stopping scroll.")
                break
            
            # 더 이상 새로운 기사가 없으면 종료
            if not new_articles_found:
                print("더 이상 새로운 기사가 없습니다. 크롤링을 종료합니다.")
                break
            
            # 스크롤 내리기: `ActionChains`를 사용해 부드럽게 스크롤 수행
            actions = ActionChains(driver)
            actions.move_to_element(footer_element).scroll_by_amount(0, -200).perform()
            time.sleep(3)  # 페이지 로딩 대기
    
    except Exception as e:
        print(f"Error during scraping: {e}")
    
    finally:
        driver.quit()
    
    return articles

# 실행 코드
if __name__ == "__main__":
    print("Cointelegraph Markets 섹션에서 기사를 크롤링 중...")
    articles = scrape_articles()
    
    print(f"총 {len(articles)}개의 기사를 수집했습니다.")
    
    # CSV 파일로 저장
    with open("cointelegraph_articles.csv", mode="w", newline="", encoding="utf-8") as file:
        fieldnames = ["title", "link", "date"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)

        writer.writeheader()  # 헤더 추가
        writer.writerows(articles)  # 모든 데이터를 한 번에 저장

    print("기사 데이터가 'cointelegraph_articles.csv' 파일에 저장되었습니다!")

Cointelegraph Markets 섹션에서 기사를 크롤링 중...
기사 저장: XRP price poised for 46% gains after Ripple secures first Dubai license (2025-03-14)
기사 저장: Crypto regulation shifts as Bitcoin eyes $105K amid liquidity boost (2025-03-13)
기사 저장: Solana price bottom below $100? Death cross hints at 30% drop (2025-03-13)
기사 저장: Bitcoin price drops 2% as falling inflation boosts US trade war fears (2025-03-13)
기사 저장: Trump family held talks with Binance for stake in crypto exchange — Report (2025-03-13)
기사 저장: Will Bitcoin price reclaim $95K before the end of March? (2025-03-13)
기사 저장: Bybit CEO on ‘brutal’ $4M Hyperliquid loss: Lower leverage as positions grow (2025-03-13)
기사 저장: 3 reasons why Ethereum can outperform its rivals after crashing to 17-month lows (2025-03-13)
기사 저장: Bitcoin must secure weekly close above $89K to confirm bottom has passed (2025-03-13)
기사 저장: Ethereum 'falling knife' warning: Is another 30% crash versus Bitcoin coming? (2025-03-13)
기사 저장: ETH/BTC hits 5-year low as trader sugges

In [42]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

def create_driver():
    """Selenium WebDriver 생성 및 설정"""
    options = webdriver.ChromeOptions()
    options.add_argument("--disable-blink-features=AutomationControlled")  # Selenium 표시 제거
    driver = webdriver.Chrome(options=options)
    return driver

def scrape_article_content(link):
    """
    특정 링크를 방문하여 기사 내용을 크롤링하는 함수.
    
    Parameters:
        link (str): 기사의 URL
    
    Returns:
        content (str): 기사의 본문 내용
    """
    driver = create_driver()
    driver.get(link)
    
    try:
        # 기사 내용 대기 및 추출
        wait = WebDriverWait(driver, 10)
        article_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "post-content")))
        content = article_element.text.strip()
        print(f"기사 내용 크롤링 완료: {link}")
    except Exception as e:
        print(f"Error while scraping content from {link}: {e}")
        content = "내용을 가져올 수 없습니다."
    finally:
        driver.quit()
    
    return content

def scrape_all_articles(input_csv, output_csv):
    """
    저장된 링크를 하나씩 방문하여 기사 내용을 크롤링하고 새로운 CSV 파일에 저장하는 함수.
    
    Parameters:
        input_csv (str): 입력 CSV 파일 경로 (제목, 링크, 날짜 포함)
        output_csv (str): 출력 CSV 파일 경로 (제목, 링크, 기사 내용 포함)
    """
    articles_with_content = []
    
    # 입력 CSV 파일 읽기
    with open(input_csv, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            title = row["title"]
            link = row["link"]
            date = row["date"]
            
            print(f"크롤링 중: {title} ({link})")
            content = scrape_article_content(link)  # 기사 내용 크롤링
            
            articles_with_content.append({
                "title": title,
                "link": link,
                "date": date,
                "content": content
            })
    
    # 출력 CSV 파일 저장
    with open(output_csv, mode="w", newline="", encoding="utf-8") as file:
        fieldnames = ["title", "link", "date", "content"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        writer.writeheader()  # 헤더 추가
        writer.writerows(articles_with_content)  # 모든 데이터를 한 번에 저장
    
    print(f"기사 데이터가 '{output_csv}' 파일에 저장되었습니다!")

# 실행 코드
if __name__ == "__main__":
    input_csv_path = "cointelegraph_articles.csv"  # 입력 CSV 파일 경로
    output_csv_path = "cointelegraph_articles_with_content.csv"  # 출력 CSV 파일 경로
    
    scrape_all_articles(input_csv=input_csv_path, output_csv=output_csv_path)

크롤링 중: XRP price poised for 46% gains after Ripple secures first Dubai license (https://cointelegraph.com/news/xrp-price-46-gains-after-ripple-secures-first-dubai-license)
기사 내용 크롤링 완료: https://cointelegraph.com/news/xrp-price-46-gains-after-ripple-secures-first-dubai-license
크롤링 중: Crypto regulation shifts as Bitcoin eyes $105K amid liquidity boost (https://cointelegraph.com/news/crypto-regulation-shifts-as-bitcoin-eyes-105-k-amid-liquidity-boost)
기사 내용 크롤링 완료: https://cointelegraph.com/news/crypto-regulation-shifts-as-bitcoin-eyes-105-k-amid-liquidity-boost
크롤링 중: Solana price bottom below $100? Death cross hints at 30% drop (https://cointelegraph.com/news/solana-price-bottom-below-100-death-cross-30-drop)
기사 내용 크롤링 완료: https://cointelegraph.com/news/solana-price-bottom-below-100-death-cross-30-drop
크롤링 중: Bitcoin price drops 2% as falling inflation boosts US trade war fears (https://cointelegraph.com/news/bitcoin-price-drops-2-falling-inflation-us-trade-war-fears)
기사 내용 크롤링 완료: http

In [43]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [1]:
import csv
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def analyze_sentiment_vader(content):
    """
    주어진 텍스트(content)의 감정을 분석하는 함수 (VADER 사용).
    
    Parameters:
        content (str): 분석할 텍스트
    
    Returns:
        sentiment (str): "Positive", "Negative", "Neutral" 중 하나
    """
    analyzer = SentimentIntensityAnalyzer()
    sentiment_scores = analyzer.polarity_scores(content)
    
    # VADER의 compound 점수를 기준으로 감정 분류
    if sentiment_scores['compound'] >= 0.05:
        return "Positive"
    elif sentiment_scores['compound'] <= -0.05:
        return "Negative"
    else:
        return "Neutral"

def perform_sentiment_analysis(input_csv, output_csv):
    """
    입력 CSV 파일에서 기사 내용을 읽어 감정 분석을 수행하고 결과를 새로운 CSV 파일에 저장하는 함수.
    
    Parameters:
        input_csv (str): 입력 CSV 파일 경로 (제목, 링크, 날짜, 내용 포함)
        output_csv (str): 출력 CSV 파일 경로 (제목, 링크, 날짜, 내용, 감정 포함)
    """
    articles_with_sentiment = []
    
    # 입력 CSV 파일 읽기
    with open(input_csv, mode="r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            title = row["title"]
            link = row["link"]
            date = row["date"]
            content = row["content"]
            
            # 감정 분석 수행
            sentiment = analyze_sentiment_vader(content)
            
            articles_with_sentiment.append({
                "title": title,
                "link": link,
                "date": date,
                "content": content,
                "sentiment": sentiment
            })
    
    # 출력 CSV 파일 저장
    with open(output_csv, mode="w", newline="", encoding="utf-8") as file:
        fieldnames = ["title", "link", "date", "content", "sentiment"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        writer.writeheader()  # 헤더 추가
        writer.writerows(articles_with_sentiment)  # 모든 데이터를 한 번에 저장
    
    print(f"감정 분석 결과가 '{output_csv}' 파일에 저장되었습니다!")

# 실행 코드
if __name__ == "__main__":
    input_csv_path = "cointelegraph_articles_with_content.csv"  # 입력 CSV 파일 경로
    output_csv_path = "cointelegraph_articles_with_sentiment.csv"  # 출력 CSV 파일 경로
    
    perform_sentiment_analysis(input_csv=input_csv_path, output_csv=output_csv_path)

감정 분석 결과가 'cointelegraph_articles_with_sentiment.csv' 파일에 저장되었습니다!


In [ ]:
import pandas as pd
import plotly.express as px

def process_sentiment_data(input_csv):
    """
    CSV 파일에서 데이터를 읽어와 날짜별 감정 비율과 강도를 처리하는 함수.
    
    Parameters:
        input_csv (str): 입력 CSV 파일 경로
    
    Returns:
        df_ratio (DataFrame): 날짜별 감정 비율 데이터프레임
        df_intensity (DataFrame): 날짜별 평균 감정 강도 데이터프레임
    """
    # CSV 파일 읽기
    df = pd.read_csv(input_csv)
    
    # 감정을 숫자로 매핑 및 가중치 적용
    sentiment_mapping = {
        "Positive": lambda x: 2 if "price rise" in x.lower() else 1.5,
        "Neutral": lambda x: 0,
        "Negative": lambda x: -2 if "price drop" in x.lower() else -1.5
    }
    
    # 각 기사 내용에 따라 가중치 적용
    df["sentiment_score"] = df.apply(
        lambda row: sentiment_mapping[row["sentiment"]](row["content"]), axis=1
    )
    
    # 날짜 형식 변환
    df["date"] = pd.to_datetime(df["date"])
    
    # 날짜별 감정 비율 계산
    df_ratio = df.groupby(["date", "sentiment"]).size().reset_index(name="count")
    total_counts = df_ratio.groupby("date")["count"].transform("sum")
    df_ratio["percentage"] = (df_ratio["count"] / total_counts) * 100
    
    # 날짜별 평균 감정 강도 계산
    df_intensity = df.groupby("date")["sentiment_score"].mean().reset_index()
    
    return df_ratio, df_intensity

def visualize_sentiment_data(df_ratio, df_intensity):
    """
    Plotly를 사용하여 날짜별 감정 비율과 강도를 시각화하는 함수.
    
    Parameters:
        df_ratio (DataFrame): 날짜별 감정 비율 데이터프레임
        df_intensity (DataFrame): 날짜별 평균 감정 강도 데이터프레임
    """
    # 감정 비율 시각화
    fig_ratio = px.bar(
        df_ratio,
        x="date",
        y="percentage",
        color="sentiment",
        title="날짜별 감정 비율",
        labels={"date": "날짜", "percentage": "비율 (%)", "sentiment": "감정"},
        barmode="stack"
    )
    
    # 평균 강도 시각화
    fig_intensity = px.line(
        df_intensity,
        x="date",
        y="sentiment_score",
        title="날짜별 평균 감정 강도",
        labels={"date": "날짜", "sentiment_score": "평균 감정 점수"},
        markers=True
    )
    
    fig_ratio.update_layout(template="plotly_white")
    fig_intensity.update_layout(template="plotly_dark")
    
    fig_ratio.show()
    fig_intensity.show()

# 실행 코드
if __name__ == "__main__":
    input_csv_path = "cointelegraph_articles_with_sentiment.csv"  # 입력 CSV 파일 경로
    
    # 데이터 처리 및 시각화
    df_ratio, df_intensity = process_sentiment_data(input_csv=input_csv_path)
    visualize_sentiment_data(df_ratio, df_intensity)